In [1]:
import pandas as pd
import spacy
import requests
from bs4 import BeautifulSoup
import PyPDF2
from neo4j import GraphDatabase
from langchain_core.runnables import (
    RunnableBranch,
    RunnableLambda,
    RunnableParallel,
    RunnablePassthrough,
)
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.prompts.prompt import PromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field
from typing import Tuple, List, Optional
from langchain_core.messages import AIMessage, HumanMessage
from langchain_core.output_parsers import StrOutputParser
import os
from langchain_community.graphs import Neo4jGraph
from langchain_community.llms import Ollama
from langchain.text_splitter import TokenTextSplitter
from langchain_experimental.graph_transformers import LLMGraphTransformer
from neo4j import GraphDatabase
from langchain_community.vectorstores import Neo4jVector

from langchain_community.vectorstores.neo4j_vector import remove_lucene_chars
from langchain_core.runnables import ConfigurableField, RunnableParallel, RunnablePassthrough
from langchain.docstore.document import Document 
'''part of the LangChain library and is used to represent a piece of text or document data along with its metadata. 
It is a common data structure used throughout the LangChain library for storing and processing text data.
In your code, you are using the Document class to create new Document objects from the preprocessed text data. 
Specifically, you are creating a dictionary with the page_content (the actual text content) and metadata (additional information about the text, such as the source), 
and then creating a Document object using that dictionary.'''
from langchain.embeddings import SentenceTransformerEmbeddings
from langchain.embeddings import HuggingFaceEmbeddings
from py2neo import Graph
from langchain_community.vectorstores import Neo4jVector
from langchain.embeddings import OpenAIEmbeddings
import os
from langchain_experimental.llms.ollama_functions import OllamaFunctions
from langchain_community.llms import Ollama
from langchain_community.chat_models import ChatOllama
from langchain.prompts import PromptTemplate
from langchain.output_parsers import StructuredOutputParser
from pydantic import BaseModel
from typing import List


In [35]:
URI = "bolt://localhost:7687"
graph = Neo4jGraph(url=URI, username="neo4j", password="password")

from langchain_community.document_loaders import TextLoader
loader = TextLoader("insta.txt")
raw_documents = loader.load()
text_splitter = TokenTextSplitter(chunk_size=512, chunk_overlap=24)
documents = text_splitter.split_documents(raw_documents[:3])

# Preprocess the documents to convert lists to tuples
'''we create a new dictionary with the page_content and metadata keys, where the metadata key has a nested dictionary with the source key. 
We convert the source value to a tuple if it's a list, and then create a new Document object from this dictionary using the Document constructor.
This way, we avoid using the non-existent to_dict and from_dict methods and create a new Document object with the desired structure.
With this modification, the code should run without the AttributeError: 'Document' object has no attribute 'to_dict' error.'''
preprocessed_documents = []
for doc in documents:
    data = {
        "page_content": doc.page_content,
        "metadata": {
            "source": tuple(doc.metadata["source"]) if isinstance(doc.metadata["source"], list) else doc.metadata["source"]
        }
    }
    preprocessed_documents.append(Document(**data))


llm = OllamaFunctions(model="llama3")
llm_transformer = LLMGraphTransformer(llm=llm)
graph_documents = llm_transformer.convert_to_graph_documents(preprocessed_documents)
graph.add_graph_documents(
    graph_documents,
    baseEntityLabel=True,
    include_source=True
)

In [36]:
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core import Settings

In [37]:


# Settings.embed_model = HuggingFaceEmbedding(
#     model_name="BAAI/bge-small-en-v1.5"
# )
from langchain_community.embeddings import OllamaEmbeddings

embeddings = OllamaEmbeddings(model="mxbai-embed-large")
vector_index = Neo4jVector.from_existing_graph(
    embeddings,
    url="bolt://localhost:7687",
    username='neo4j',
    password='password',
    search_type="hybrid",
    node_label="Document",
    text_node_properties=["text"],
    embedding_node_property="embedding"
)
# Retriever

graph = Graph("bolt://localhost:7687", auth=("neo4j", "password"))

graph.run(
    "CREATE FULLTEXT INDEX entity IF NOT EXISTS FOR (e:__Entity__) ON EACH [e.id]"
)

# Extract entities from text
class Entities(BaseModel):
    """Identifying information about entities."""

    names: List[str] = Field(
        ...,
        description="All the person, organization, or business entities that "
        "appear in the text",
    )

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are extracting organization and person entities from the text.",
        ),
        (
            "human",
            "Use the given format to extract information from the following "
            "input: {question}",
        ),
    ]
)



In [38]:
from pyvis.network import Network

def showGraph(graph_documents):
    net = Network(height="800px", width="100%", bgcolor="#222222", font_color="white")
    
    nodes = graph_documents[0].nodes
    relationships = graph_documents[0].relationships
    
    for node in nodes:
        net.add_node(node.id, label=node.id, title=str(node.type), color="skyblue")
    
    for relationship in relationships:
        net.add_edge(relationship.source.id, relationship.target.id, title=relationship.type, color="gray", arrows="to")
    
    net.repulsion()
    
    # Generate HTML file
    net.write_html("graph.html")

In [39]:
showGraph(graph_documents)

In [40]:
entity_chain = prompt | llm.with_structured_output(Entities)

In [41]:
from langchain_community.chat_models import ChatOllama

llm = ChatOllama(model="llama3", format="json", temperature=0)

In [42]:
messages = [
    AIMessage(
        content="tell me about instagram"
    )
]

chat_model_response = llm.invoke(messages)
print(chat_model_response.content)

{



}
